<a href="https://colab.research.google.com/github/lacykaltgr/continual-learning-ait/blob/experiment/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''Download the files '''
'''Only for colab'''

!wget https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/main.zip
!unzip main.zip
!find continual-learning-ait-main -type f ! -name "main.ipynb" -exec cp {} . \;

!rm -r stable_diffusion
!mkdir stable_diffusion
!mv diffusion_model.py stable_diffusion/
!mv autoencoder_kl.py stable_diffusion/
!mv layers.py stable_diffusion/
!mv stable_diffusion.py stable_diffusion/
!mv constants.py stable_diffusion/

--2023-05-10 12:40:01--  https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/main [following]
--2023-05-10 12:40:02--  https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ] 338.93K  --.-KB/s    in 0.05s   

2023-05-10 12:40:02 (6.55 MB/s) - ‘main.zip’ saved [347065]

Archive:  main.zip
4e98f3b1d72a496ef351c2ffa7a12a2d9dddf2d8
   creating: continual-learning-ait-main/
  i

In [12]:
import tensorflow as tf
import keras
from stable_diffusion.layers import PaddedConv2D
from stable_diffusion.autoencoder_kl import ResnetBlock, AttentionBlock
from classifier import GatedDense
from data_preparation import load_dataset, CLDataLoader
from keras.callbacks import EarlyStopping
from keras import layers
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Activation, Flatten, BatchNormalization, Conv2DTranspose, UpSampling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator

In [3]:
def load_cifar_10():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    n_classes = 10
    X_train = (X_train / 127.5) -1
    X_test = (X_test / 127.5) -1
    y_train = tf.keras.utils.to_categorical(y_train, n_classes)
    y_test = tf.keras.utils.to_categorical(y_test, n_classes)
    return (X_train, y_train), (X_test, y_test)

In [4]:
dpt_train, dpt_test = load_dataset('cifar-10', n_classes_first_task=4, n_classes_other_task=3)
batch_size = 512
train_loader = CLDataLoader(dpt_train, batch_size , train=True)
test_loader = CLDataLoader(dpt_test, batch_size, train=False)
INPUT_SHAPE = (32, 32, 3)
KERNEL_SIZE = (3, 3)

170498071/170498071 [==============================] - 9s 0us/step


#Encoder

In [13]:
encoder = keras.Sequential()

encoder.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
encoder.add(BatchNormalization())
encoder.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
encoder.add(BatchNormalization())
encoder.add(MaxPool2D(pool_size=(2, 2)))
encoder.add(Dropout(0.25))

encoder.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
encoder.add(BatchNormalization())
encoder.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
encoder.add(BatchNormalization())
encoder.add(MaxPool2D(pool_size=(2, 2)))
encoder.add(Dropout(0.25))

encoder.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
encoder.add(BatchNormalization())
encoder.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
encoder.add(BatchNormalization())
encoder.add(MaxPool2D(pool_size=(2, 2)))

encoder.add(keras.layers.GroupNormalization(epsilon=1e-5)),
encoder.add(keras.layers.Activation("swish")),
encoder.add(ZeroPadding2D((1, 1)))
encoder.add(Conv2D(16, 3, strides=(1, 1)))
encoder.add(ZeroPadding2D((1, 1)))
encoder.add(Conv2D(8, 1, strides=(1, 1)))

In [14]:
print("#trainable parameters:",encoder.count_params())

#trainable parameters: 307640


#Classifier

In [15]:
classifier = keras.Sequential()

classifier.add(Flatten())
classifier.add(Dropout(0.2))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dropout(0.25))
classifier.add(Dense(10, activation='softmax'))

#Decoder

In [16]:
decoder = keras.Sequential()


decoder.add(Conv2DTranspose(filters=128, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(filters=128, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
decoder.add(BatchNormalization())
decoder.add(UpSampling2D(size=(2, 2)))

decoder.add(Conv2DTranspose(filters=64, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(filters=64, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
decoder.add(BatchNormalization())
decoder.add(UpSampling2D(size=(2, 2)))

decoder.add(Conv2DTranspose(filters=32, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
decoder.add(BatchNormalization())
decoder.add(Conv2DTranspose(filters=32, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
decoder.add(BatchNormalization())
decoder.add(UpSampling2D(size=(2, 2)))

decoder.add(keras.layers.GroupNormalization(epsilon=1e-5)),
decoder.add(keras.layers.Activation("swish")),
decoder.add(Conv2DTranspose(16, 3, padding='same' )),
decoder.add(Conv2DTranspose(3, 1, activation='sigmoid'))

In [17]:
input_image = keras.Input(shape=(32, 32, 3), name="image")
encoded = encoder(input_image)
decoded = decoder(encoded)
classified = classifier(encoded)


In [18]:
#encoder_decoder = keras.Model(inputs = input_image, outputs = decoded)
#encoder_decoder.compile(
#    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
#    loss='mse',
#)

In [19]:
encoder_classifier = keras.Model(inputs = input_image, outputs = classified)
encoder_classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-3),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [20]:

es1 = EarlyStopping(patience=1, monitor="val_loss")
es2 = EarlyStopping(patience=10, monitor="val_loss", restore_best_weights=True)

In [ ]:
#encoder_decoder.fit(X_train, X_train, batch_size=64, epochs=3, validation_data=(X_test, X_test), callbacks=[es1])

#Training

In [ ]:
prev_test = None
for (tr_loader, ts_loader) in zip(train_loader, test_loader):
  for epoch in range(5):
    for ((data, target) ,(testdata, testtarget)) in zip(tr_loader, ts_loader):
      encoder_classifier.fit(data, target, epochs=1, validation_data=(testdata, testtarget), callbacks=[es2])
  if (prev_test is not None):
    for (testdata, testtarget) in prev_test:
        encoder_classifier.evaluate(testdata, testtarget)
  prev_test = ts_loader
  print('\n\nNew Task')

5/5 [==============================] - 0s 21ms/step - loss: 0.3012 - accuracy: 0.8816 - val_loss: 0.2849 - val_accuracy: 0.8750


New Task
2/2 [==============================] - 0s 8ms/step - loss: 17.5139 - accuracy: 0.0000e+00


New Task
1/1 [==============================] - 0s 23ms/step - loss: 28.2109 - accuracy: 0.0000e+00


New Task


#Export the model

In [21]:
(X_train, y_train), (X_test, y_test) = load_cifar_10()
encoder_classifier.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[es2])

Epoch 1/100
1563/1563 [==============================] - 25s 10ms/step - loss: 1.5032 - accuracy: 0.4556 - val_loss: 1.1502 - val_accuracy: 0.6000
Epoch 2/100
1563/1563 [==============================] - 12s 8ms/step - loss: 1.0521 - accuracy: 0.6321 - val_loss: 0.9515 - val_accuracy: 0.6714
Epoch 3/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.9039 - accuracy: 0.6931 - val_loss: 0.8167 - val_accuracy: 0.7318
Epoch 4/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.8048 - accuracy: 0.7266 - val_loss: 0.7245 - val_accuracy: 0.7551
Epoch 5/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.7406 - accuracy: 0.7502 - val_loss: 0.6519 - val_accuracy: 0.7798
Epoch 6/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.6974 - accuracy: 0.7652 - val_loss: 0.6192 - val_accuracy: 0.7918
Epoch 7/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.6485 - accuracy: 0.7827 - val_loss: 0.6282 

In [22]:
encoder.save("encoder.h5")
classifier.save("classifier.h5")